$$Authors: Toke~ Faurby~ (s136232)~ and~ Maciej~ Korzepa~ (mjko).$$

#### Important notes
 * illustrate the knowledge of the Bayesian workflow.
 * Structure: The notebook presents a clear cohesive data analysis story, which is enjoyable to read
 * Statistical terms: Statistical terms are used accurately and with clarity

#### TODO:
 * Discussion of problems, and potential improvements
 * Conclusion:
     * The main conclusion of the data analysis should be clear
     
 * Convergence diagnostics (Rhat, divergences, neff)
 * Model comparison if applicable (e.g. with loo)
 * Posterior predictive checking
 * Predictive performance assessment if applicable (e.g. classification accuracy)
 * Potentially sensitivity analysis



#### DONE: 
 * Introduction: 
     * The introduction is inviting, presents an overview of the notebook. Information is relevant and presented in a logical order.
 * Description of the data
 * Description of analysis problem
 * Description of Frequentis UCB
 * Description of Bayesian UCB
 * Description of Thompson Sampling
 * Description of Hierarchical extensions
 * Description of the prior
 * Inlude code


Things we skip
 * Stan Code 
 * How Stan model is run

# Introduction
**Reinforcement learning** is about goal directed learning from interactions, and is generally thought of as distinct from supervised and unsupervised learning.
The goal is to learn what actions to take, and when to take them, so as to optimize long-term performance.
This may involve sacrificing immediate reward to obtain greater reward in the long-term or just to obtain more information about the environment.
The tradeoff between maximizing reward and learning about the environment is called the _exploration/exploitation dillemma_, and it is one of the problems at the core of reinforcement learning.


**Bayesian methods** uses Bayes rule to define a posterior distribution, based on a prior on the model parameters and the likelihood of the obsevations given model parameters.
Should we obtain even more observations, the old posterior becomes the new prior and the process is repeated.
This property makes Bayesian models attractive for reinforcement learning, as the ability to effectively utilize new observations as they become available is important for overall performance (unlike many other data analysis problems).
This explicit modelling of probabilities gives rise to principled methods for incorporating prior information and action-selection (exploration/exploitation) as a function of the uncertainty in learning.
Bayesian methods also enable hierarchical approaches, that enhace data efficiency even further.


In this notebook we will show the principles of **Bayesain reinforcement learning** using the classic and very simple problem - the **multi-armed bandit problem**.
At each time step the agent must select one of $K$ arms, and will subsequently recieve a reward based on some unknown probability distribution $p(\theta_k)$.
The goal is to get as high a total reward as possible.
The performance is often measured in terms of expected regret, i.e. the difference between the selected actions, and the optimal action.

$$
\mathbb{E} \left(Regret(T)\right) 
=
\mathbb{E} \left[ \sum_{t=1}^T \left( r(a^*) - r(a_t) \right)\right]
$$

where $r(a)$ is the recieved reward after performing action $a$. 
$a^*$ is the action that gives the highest expected reward.

The environment is therefore static, greatly simplifying the problem, but many of the general properties still hold.


## Content
1. The Environment: Multi-Armed Bernoulli Bandits
* The Agents - Frequentist baseline, Bayesian approach, and hierarchical Bayesian approach
* Implementation - what we did and how it works
* Results
* Conclusion and discussion

# The Environment: Multi-Armed Bandits (K-MAB)

The **data** is obtained iteratively through interacting with the environment, which we define as follows:

When initialized $K$ parameters $\theta_k$ are sampled from a $Beta(\alpha, \beta)$ distribution, with $\alpha=10$ and $\beta=40$ (these values are arbitrary, but we don't want to use a uniform distribution, as it would then match our prior).
One episode lasts $1000$ steps, and at each time step, $t$ the agent picks an action, $a_t$, and recieves a reward, sampled from a Bernoulli distribution, with parameter $\theta_{a_t}$.

The agent must therefore balance selecting arms that it believes are good (exploit high expected $\theta_k$) and unknown arms that might be even better (explore uncertain $\theta_k$).



> **NB:** Two files `agents.py` and `bandit.py` hold the code describing the agents and bandit environment respectively. The code is reproduced at the end of the notebook (in case you want to run it, run those first). The code is also available [here](https://github.com/Faur/Multi-Armed-Bayesian-Bandits). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

from multiprocessing import Pool

In [ ]:
try:
    from bandit import *
except:
    print('Download the code from github: https://github.com/Faur/Multi-Armed-Bayesian-Bandits')
    print('or run the code block in the appendix')

# The Agents
In order to demonstrate the effectiveness of Bayesian approaches we use the popular frequentist *upper confidence bound* (UCB) method as a benchmark.
For Bayesian approaches we have selected '*Bayesian UCB*', for easy direct comparison, and '*Thompson Sampling*' (aka. probability matching), which achieves state of the art performance on the $k=2$ task.
We also examin the hierarchical extensions of the two Bayesian approaches.


## Frequentist UCB
The idea behind frequentist UCB is that we should always explore to some extent, as we can never be certain whether or not we have foun the optimal arms.
Exploratory actions should be selected based on their potential for being optimal, taking into account both how close their estimates are to being optimal and the uncertainties in those estimates.
This is done through optimistic action selection, using the following selection criteria:

$$
a_t = \arg \max_a \left[ \overline{r(a)} + c\sqrt{\frac{\ln{t}}{N(a)}} \right]
$$

Where $\overline{r(a)}$ is the emperical mean reward recieved from performing action $a$, $c$ is a hyperparameter determining the confidence level, and $N(a)$ returns the number of times action $a$ has been selected.

Each time $a$ is selected the associated is reduced ($N(a)$ increases).
Similarly when an action other than $a$ is selected  the uncertainty estimate increases ($t$ increases)
The natural logarithm insures that the means that this increase in uncertainty gets smaller over time, but it is still unbounded, meaning that in the limit all actions will be taken infinitely many times (constant exploration).




## The Bayesian Approach
According to K-MAB model, the only unknown quantities are action dependent outcome probabilities $P(\cdot | a)$. 
We can learn these probabilities using Bayesian interference during sequential interaction with the MAB.
For our Bernoulli K-MAB model, action outcome probabilities are parametrized by vector $\mathbf{\theta} \in [0,1]^{K}$.
As each interaction with the model corresponds to performing a Bernoulli trial, the natural choice of prior for $\mathbf{\theta}$ is Beta distribution for each arm $a$ i.e. 

$$p(\theta_a) \sim Beta(\alpha_a,\beta_a)$$

The prior for $\theta_a$ is chosen to be a non-informative prior $Beta(1,1)$, as we don't have any prior knowledge about possible values of $\theta_a$.
As this is also conjugate prior for bernoulli distribution, we can easily calculate the posterior of $\theta_a$ given one observation:

$$p(\theta_a|y) \propto Beta(\alpha_a+y,\beta_a+1-y)$$

where $y\in\{0,1\}$ is the reward.
Whenever we draw an arm and observe a reward (whenever we perform a Bernoulli trial), we update the posterior and use it as prior for the following draws from that arm.

We can, however, take a different (but equivalent) approach where we keep the prior fixed and calculate the posterior using binomial likelihood for $n_a$ trials and $y_a$ successes for arm $a$.
The posterior update step looks then as follows:

$$p(\theta_a|y_a) \propto Beta(\alpha+y_a,\beta+n_a-y_a)$$

for prior $Beta(\alpha, \beta)$.
We use this latter approach in our implementation as it will later provide a convenient basis to extend the model with hierarchical prior.

The main advantage of such a Bayesian framework is that it allows to quantify the uncertainty about $\mathbf{\theta}$ which would not be possible using frequentist approach.
The posterior distribution of $\mathbf{\theta}$, not just a point estimate, gives a basis to guide the exploration using different approaches.
In this project, we focus on two of them: Bayesian Upper Confidence Bound (Bayesian UCB) and Thompson sampling.


## Bayesian UCB

Bayesian UCB is, as the name implies, a Bayesian version of frequentist UCB and was proposed by Kaufmann et al[1].
For each arm $a$, we obtain the value of a specific quantile, $q_a$: 

$q_a=Q(1-\frac{1}{t(\log n)^c}, \theta_a)$

where $(\log n)^c$ is an artefact of the theoretical analysis, but the authors achieved most satisfying results with $c=0$ which simplifies the expression to:

$q_a=Q(1-\frac{1}{t},\theta_a)$.

Then, we draw the arm $a$ that has the highest value of $q_a$. After observing a new reward, we update the posterior distribution of $\theta_a$.

The fact that we use a high quantile, rather than the expectation ensures exploration, and is similar to the optimistic action selection that is used in frequentist UCB.

## Thompson sampling

Thompson sampling is a heurestic that uses the prior distribution to address the exploration-exploitation dilemma in a natural way. 
It choses the action that maximizes the expected reward with respect to a randomly drawn belief.
In our K-MAB problem, we draw a sample $\hat{\mathbf{\theta}}$ from the posterior distribution $\mathbf{\theta}$ and select the optimal action with respect to the model defined by $\hat{\mathbf{\theta}}$.


# Hierarchical Bayesian K-MAB
> TODO: Only use point estimates?

In K-MAB problem, individual $\theta_a$ probabilities are often drawn from a specific distribution, as is the case in our formulation.
In the separate model presented above, we do not utilize this knowledge.
However, if we use hierarchical approach, we could model the source distribution that generates $\theta_a$ probabilities.
In this project, for simplicity, we assume that we know that $\theta_a$ probabilities are generated from Beta distribution($\theta_a \propto Beta(\alpha,\beta)$). To estimate $\alpha$ and $\beta$ parameters, we can use the distribution mean $\mu$ and variance $\sigma^2$:

$$\mu = \frac{\alpha}{\alpha + \beta}$$

$$\sigma^2 = \frac{\alpha\beta}{(\alpha + \beta)^2(\alpha+\beta+1)}$$

Solving these equations for $\alpha$ and $\beta$ gives:

$$\beta = \frac{\mu(1-\mu)^2}{\sigma^2}-(1-\mu)$$

$$\alpha = \frac{\mu \beta}{1-\mu}$$

We calculate $\mu$ and $\sigma^2$ as follows:

$$\mu = E(\theta)$$

$$\sigma^2 = Var(\theta)$$

where $\mathbf{\theta}$ is a vector with elements $\frac{y_a}{n_a}$ for all $a \in [1..K]$.

We must note that this is not a fully Bayesian approach as $\mathbf{\theta}$ is a point estimate. A fully Bayesian approach would require numerical approximation to compute the posterior and as Stan does not yet allow to use posterior samples as prior for the new data (https://groups.google.com/forum/#!topic/stan-users/b0MWu4GJygI), we decided to build our prior only based on point estimate.

We recalculate the hierarchical prior every time we pull an arm and observe a reward. The posterior is always calculated using the updated prior. Such a model is compatible with Bayesian UCB and Thompson sampling without any further modifications.



In [ ]:
try:
    from agents import *
except:
    print('Download the code from github: https://github.com/Faur/Multi-Armed-Bayesian-Bandits')
    print('or run the code block in the appendix')

# Experiments

> TODO: Can we get stan to work? http://andrewgelman.com/2018/02/04/andrew-vs-multi-armed-bandit/

In the code below we run and compare the following agents:
 * Random agent 
 * Frequentist UCB
 * Bayesian UCB 
 * Hierarchical Bayesian UCB 
 * Thompson Sampling
 * Hierarchical Thompson Sampling

for $k = \{2, 4, 8, 16, 32\}$.
The for each $k$ the experiment is repeated 500 times, and the mean regret is plotted.
(Standard deviations have been left out of the plots, as they make the plots cluttered.)

In [ ]:
## Helper functions

def make_agents(env, k, max_steps):
    agents = [OptimalAgent(env), RandomAgent(k), FreqUCB(k), BayesUCB(k, max_steps), HierarchicalBayesUCB(k, max_steps), ThompsonSampling(k, max_steps), HierarchicalThompsonSampling(k, max_steps)]
    return agents

def run(args):
    """ Run all the agnets in 'make_agents' once."""
    k, max_steps, alpha, beta = args 
    env = KBandit(k, max_steps=max_steps, alpha=alpha, beta=beta)
    agents = make_agents(env, k, max_steps)

    rewards = []
    _, _, d, _ = env.reset()
    for a in agents:
        a.reset()
    while not d:
        draw = True
        rewards_ = []
        for agent in agents:
            a = agent.action()
            _, r, d, _ = env.step(a, draw)
            draw = False
            agent.update(a, r)
            rewards_.append(r)
        rewards.append(rewards_)

    return np.array(rewards).T

def runs(num_runs, args, mp=False):
    """ Run num_runs experiments, either sequentially or using multiprocessing."""
    try:
        if mp:
            print('multiprocessing')
            proc_pool = Pool()
            rewards = proc_pool.map(run, [args for i in range(num_runs)])
        else:
            print('single thread')
            rewards = []
            for i in range(num_runs):
                print('\rrun', i+1, 'of', num_runs, end=''); sys.stdout.flush()
                rewards.append(run(args))
            print()
    except KeyboardInterrupt:
        pass

    rewards = np.array(rewards)
    # rewards = np.mean(rewards, 0)
    return rewards


def plot_cumsum(rewards, agents_names, use_std=False):
    """ Plotting helper function."""
    colors=plt.cm.rainbow(np.linspace(0,1,rewards.shape[1]))
    plt.figure(figsize=(6,6))
    for i in range(len(agents_names)-1):
    #     plt.plot(np.cumsum(rewards[0] - rewards[i+1]), label=agents_names[i+1]+' regret')
        c = colors[i]
        cumsums = np.cumsum(rewards[:,0,:] - rewards[:,i+1,:], -1)
        means = np.mean(cumsums, 0)
        stds = np.std(cumsums, 0)
        plt.plot(means, c=c, lw=2, label=agents_names[i+1], alpha=0.75)
        if use_std:
            plt.plot(means+stds, c=c, linestyle='--', alpha=0.5)
            plt.plot(means-stds, c=c, linestyle='--', alpha=0.5)

    plt.legend()

> **NB:** [Multiprocessing is strange on **Windows**](https://docs.python.org/2/library/multiprocessing.html#windows), and can therefore not be run inside a Jupyter Notebook. Use `useMultiProcess = False`, or run the `run.py` file from github.

In [ ]:
1/0

In [ ]:
## run.py

show_visualization = True
save_rewards = False
useMultiProcess = True

max_steps = 500
num_episodes = 500

ks = [2, 4, 8, 16, 32]

print('save_rewards', save_rewards)
print('max_steps', max_steps)
print('num_episodes', num_episodes)
print('ks', ks)
print()

agents_names = ['Optimal', 'Random', 'FreqUCB', 'BayesUCB', 'HierarchicalBayesUCB', 'ThompsonSampling', 'HierarchicalThompsonSampling']
for k in ks:
    print('k =',k)
    alpha = 10; beta = 40
    args = (k, max_steps, alpha, beta)

    rewards = runs(num_episodes, args, mp=useMultiProcess)
    if save_rewards:
        save_name = 'raw_rewards_'+str(k)+'.npy'
        print('saving', save_name)
        np.save(save_name, rewards)

    plot_cumsum(rewards, agents_names)
    plt.title('Regret, k='+str(k))
    plt.draw()

    plot_cumsum(rewards, agents_names, True)
    plt.title('Regret, k='+str(k))
    plt.draw()
    print()

if show_visualization:
    plt.show()

print("Done")

# Results

The experiments take quite a long time to finish, and were therefore run on the DTU servers.
The results (as reported by `plot_cum_sum`) are reproduced here.

![](./img/collection.png)

Looking at these pictures, we can observe several things about the performance.
1. As $k$ increases the problem becomes more difficult, as indicated by the increasing rate of regret.
This is becaues there are more arms to test, and the difference between a random arm, and the best arm becomes larger.
This makes convergence to take longer, and we can clearly see that for the higer values of $k$ the agents haven't fully converged after 500 steps.
(Due to time constraints we were unable to run the model for longer.)

* Frequentist UCB is consistently better than random and worse than the Bayesian approaches.
* As $k$ increases the benifit of the hierarchical approach becomes more and more apparent, as it is able to learn quicker, by generalizing between arms.


# COMMENTS AND NOTES

Posterior predictive checking
* Does this make sense in BRL? We are not really interested in accurately monitoring the world, rather we are interested in finding the best action
* Perhaps determining the accuracy on the most frequent action is a good idea?
* in hierarchical, does the shared prior end up matching a=10, b=40?

# Conclusion and discussion
 * The main conclusion of the data analysis should be clear

# References

[1] Kaufmann, E., Garivier, A., & Paristech, T. (2013). On bayesian upper confidence bounds for bandit problems. https://doi.org/10.1.1.392.8327

# Appendix

In [ ]:
### agents.py

In [ ]:
### bandit.py